# Preprocessing Pandas

In [ ]:
import pandas as pd
sample_submission = pd.read_csv("understanding_cloud_organization/sample_submission.csv")
train = pd.read_csv("understanding_cloud_organization/train_320.csv")

In [ ]:
train['Image_name'] = train['Image_Label'].apply(lambda x: x.split('_')[0])
train['Label_name'] = train['Image_Label'].apply(lambda x: x.split('_')[1])
train.drop('Image_Label',axis=1,inplace=True)
train = train.pivot('Image_name','Label_name','EncodedPixels')

In [ ]:
train.head()

# Data Augmentation

In [1]:
import os
import pandas as pd
from skimage.io import imread
df = pd.DataFrame(columns = ['Name' , 'Image'])
directory = "understanding_cloud_organization/train_images_320"

for image_name in os.listdir(directory) :
    if image_name.split(".")[1]=='jpg':
        im = imread(directory + '/' + image_name)
        df = df.append({'Name' : image_name.split(".")[0], 'Image': im}, ignore_index=True)
        
        

In [2]:
df_mask = pd.DataFrame(columns = ['Name' , 'Image'])
directory = "understanding_cloud_organization/train_images_320_Black_Fish_Gravel_Sugar_Flower"

for image_name in os.listdir(directory) :
    if image_name.split(".")[1]=='jpg':
        im = imread(directory + '/' + image_name)
        df_mask = df_mask.append({'Name' : image_name.split(".")[0], 'Image': im}, ignore_index=True)

Test 1 augmentation data

In [3]:


for i, im in tqdm(enumerate(df["Image"])) :
    image_name = df["Name"].iloc[i]
    for da in range(1, 8+1) :
        transform = augm.Compose([
            augm.RandomRotate90(p=1), # Rotate 90°
            augm.HorizontalFlip(p=1), # Horizontal Flip, to keep brain structure (don't do vertical flip)
            augm.CLAHE(p=1), # Apply Contrast Limited Adaptive Histogram Equalization to the input image
            augm.ColorJitter(p=1), # Randomly changes the brightness, contrast, and saturation of an image
            augm.OpticalDistortion(p=1),
            augm.GridDistortion(p=1),
            augm.HueSaturationValue(p=1) # Randomly change hue, saturation and value of the input image
        ])
        transformed = transform(image=im)
        transformed_image = transformed["image"]    
        df = df.append({'Name': image_name + "_da" + str(da), 'Image': transformed_image}, ignore_index=True)

5546it [05:47, 15.96it/s]


In [4]:
for i, im in tqdm(enumerate(df_mask["Image"])) :
    mask_name= df_mask["Name"].iloc[i]
    for da in range(1, 8+1) :
        transform = augm.Compose([
            augm.RandomRotate90(p=1), # Rotate 90°
            augm.HorizontalFlip(p=1), # Horizontal Flip, to keep brain structure (don't do vertical flip)
            augm.CLAHE(p=1), # Apply Contrast Limited Adaptive Histogram Equalization to the input image
            augm.ColorJitter(p=1), # Randomly changes the brightness, contrast, and saturation of an image
            augm.OpticalDistortion(p=1),
            augm.GridDistortion(p=1),
            augm.HueSaturationValue(p=1) # Randomly change hue, saturation and value of the input image
        ])
        transformed= transform(image=im)  
        transformed_mask = transformed["image"]
        df_mask = df_mask.append({'Name': mask_name + "_da" + str(da), 'Image': transformed_mask}, ignore_index=True)

5546it [06:25, 14.40it/s]


Test 2 Augmentation data

In [4]:
import albumentations as augm
from tqdm import tqdm

In [14]:
for i, im in tqdm(enumerate(df["Image"])) :
    image_name = df["Name"].iloc[i]
    transformrot90 = augm.RandomRotate90(p=1) # Rotate 90° 
    transformHorizontalFlip= augm.HorizontalFlip(p=1) # Horizontal Flip
    transfDistorsion = augm.OpticalDistortion(p=1)
    transfOpticalDistortion= augm.OpticalDistortion(p=1)

    transformed1 = transformrot90(image=im)
    transformed2 = transformHorizontalFlip(image=im)
    transformed3=transfDistorsion(image=im)
    transformed4=transfOpticalDistortion(image=im)

    transformed_image1 = transformed1["image"]    
    transformed_image2 = transformed2["image"]  
    transformed_image3 = transformed3["image"]  
    transformed_image4 = transformed4["image"]  

    df = df.append({'Name': image_name + "_transformrot90" , 'Image': transformed_image1}, ignore_index=True)
    df = df.append({'Name': image_name + "_transformHorizontalFlip", 'Image': transformed_image2}, ignore_index=True)
    df = df.append({'Name': image_name + "_transfDistorsion" , 'Image': transformed_image3}, ignore_index=True)
    df = df.append({'Name': image_name + "_transfOpticalDistortion" , 'Image': transformed_image4}, ignore_index=True)

5546it [02:35, 35.62it/s]


In [5]:
for i, im in tqdm(enumerate(df_mask["Image"])) :
    image_name = df_mask["Name"].iloc[i]
    transformrot90 = augm.RandomRotate90(p=1) # Rotate 90° 
    transformHorizontalFlip= augm.HorizontalFlip(p=1) # Horizontal Flip
    transfDistorsion = augm.OpticalDistortion(p=1)
    transfOpticalDistortion= augm.OpticalDistortion(p=1)

    transformed1 = transformrot90(image=im)
    transformed2 = transformHorizontalFlip(image=im)
    transformed3=transfDistorsion(image=im)
    transformed4=transfOpticalDistortion(image=im)

    transformed_image1 = transformed1["image"]    
    transformed_image2 = transformed2["image"]  
    transformed_image3 = transformed3["image"]  
    transformed_image4 = transformed4["image"]  

    df_mask = df_mask.append({'Name': image_name + "_transformrot90" , 'Image': transformed_image1}, ignore_index=True)
    df_mask = df_mask.append({'Name': image_name + "_transformHorizontalFlip", 'Image': transformed_image2}, ignore_index=True)
    df_mask = df_mask.append({'Name': image_name + "_transfDistorsion" , 'Image': transformed_image3}, ignore_index=True)
    df_mask = df_mask.append({'Name': image_name + "_transfOpticalDistortion" , 'Image': transformed_image4}, ignore_index=True)

5546it [01:00, 91.64it/s]


In [6]:
from PIL import Image
save = True 

In [17]:

if(save):
    for i, img in tqdm(enumerate(df["Image"])):
        im = Image.fromarray(img)
        im.save(os.path.join('understanding_cloud_organization/augmented_data2/', df["Name"].iloc[i] + ".jpg"))


27730it [06:15, 73.80it/s]


In [7]:

if(save):
    for i, img in tqdm(enumerate(df_mask["Image"])):
        im = Image.fromarray(img)
        im.save(os.path.join('understanding_cloud_organization/augmented_data2_mask/', df_mask["Name"].iloc[i] + ".jpg"))

27730it [04:14, 109.07it/s]


# Affichage Image

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

img=mpimg.imread('./understanding_cloud_organization/train_images_320/009e2f3.jpg')
img2=mpimg.imread('./understanding_cloud_organization/train_images_320_Black_Fish_Gravel_Sugar_Flower/009e2f3.jpg')

f, axarr = plt.subplots(1,2)
axarr[0].imshow(img)
axarr[1].imshow(img2)



# DATA LOADING

In [ ]:
from transformations import MoveAxis, Normalize01

In [ ]:
from transformations import Compose, DenseTarget
from transformations import MoveAxis, Normalize01
from customdatasets import SegmentationDataSet
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import pathlib

# root directory
root = pathlib.Path.cwd() / 'understanding_cloud_organization'
def get_filenames_of_path(path: pathlib.Path, ext: str = '*'):
    """Returns a list of files in a directory/path. Uses pathlib."""
    filenames = [file for file in path.glob(ext) if file.is_file()]
    return filenames

# input and target files
inputs = get_filenames_of_path(root / 'train_images_320')
targets = get_filenames_of_path(root / 'train_images_320_Black_Fish_Gravel_Sugar_Flower')
    

# training transformations and augmentations
transforms = Compose([
    DenseTarget(),
    Normalize01(),
    MoveAxis(),
])

# random seed
random_seed = 42

# split dataset into training set and validation set
train_size = 0.8  # 80:20 split

inputs_train, inputs_valid = train_test_split(
    inputs,
    random_state=random_seed,
    train_size=train_size,
    shuffle=True)


targets_train, targets_valid = train_test_split(
    targets,
    random_state=random_seed,
    train_size=train_size,
    shuffle=True)

# dataset training
dataset_train = SegmentationDataSet(inputs=inputs_train,
                                    targets=targets_train,
                                    transform=transforms)

# dataset validation
dataset_valid = SegmentationDataSet(inputs=inputs_valid,
                                    targets=targets_valid,
                                    transform=transforms)

# dataloader training
dataloader_training = DataLoader(dataset=dataset_train,
                                 batch_size=1,
                                 shuffle=False)
# dataloader validation
dataloader_validation = DataLoader(dataset=dataset_valid,
                                   batch_size=1,
                                   shuffle=False)



In [ ]:
x, y = next(iter(dataloader_training))
print(f'x = shape: {x.shape}; type: {x.dtype}')
print(f'x = min: {x.min()}; max: {x.max()}')
print(f'y = shape: {y.shape}; class: {y.unique()}; type: {y.dtype}')
print(f'y = min: {y.min()}; max: {y.max()}')


## MODEL UNET

In [ ]:
import torch
from unet import *
model = UNet(in_channels=3,
             out_channels=4,
             n_blocks=2,
             start_filters=32,
             activation='relu',
             normalization='batch',
             conv_mode='same',
             dim=2).cuda()

x = torch.randn(size=(1, 3, 320, 480), dtype=torch.float32).cuda()

with torch.no_grad():
    out = model(x)

print(f'Out: {out.shape}')

In [ ]:
from torchsummary import summary
summary = summary(model, (3, 320, 480))



In [ ]:
shape = 1920


def compute_max_depth(shape, max_depth=10, print_out=True):
    shapes = []
    shapes.append(shape)
    for level in range(1, max_depth):
        if shape % 2 ** level == 0 and shape / 2 ** level > 1:
            shapes.append(shape / 2 ** level)
            if print_out:
                print(f'Level {level}: {shape / 2 ** level}')
        else:
            if print_out:
                print(f'Max-level: {level - 1}')
            break

    return shapes


out = compute_max_depth(shape, print_out=True, max_depth=10)


In [ ]:
low = 10
high = 512
depth = 8


def compute_possible_shapes(low, high, depth):
    possible_shapes = {}
    for shape in range(low, high + 1):
        shapes = compute_max_depth(shape,
                                   max_depth=depth,
                                   print_out=False)
        if len(shapes) == depth:
            possible_shapes[shape] = shapes

    return possible_shapes
possible_shapes = compute_possible_shapes(low, high, depth)
possible_shapes


# TRAIN ET RUN

In [ ]:
from train import Trainer

# device
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    torch.device('cpu')

# criterion
criterion = torch.nn.BCEWithLogitsLoss()

# optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# trainer
trainer = Trainer(model=model,
                  device=device,
                  criterion=criterion,
                  optimizer=optimizer,
                  training_DataLoader=dataloader_training,
                  validation_DataLoader=dataloader_validation,
                  lr_scheduler=None,
                  epochs=4,
                  epoch=0,
                  notebook=True)

# start training
training_losses, validation_losses, lr_rates = trainer.run_trainer()


# SAVE THE MODEL

In [ ]:
model_name =  'cloud.pt'
torch.save(model.state_dict(), pathlib.Path.cwd() / model_name)

# LEARNING RATE

In [ ]:
from lr_rate_finder import *
lrf = LearningRateFinder(model, criterion, optimizer, device)
lrf.fit(dataloader_training, steps=1000)

In [ ]:
lrf.plot()


In [ ]:
fig = plt.show(training_losses, validation_losses)
print(training_losses)
print(validation_losses)

In [ ]:
from visual import plot_training
fig = plot_training(training_losses, validation_losses, lr_rates, gaussian=True, sigma=1, figsize=(10, 4))